<a href="https://colab.research.google.com/github/rtikyani/CS634_IBM_Fairness_Pipeline/blob/master/CS634_Project1_Team3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1

Read the paper and fully understand the fairness pipeline. Write your own tutorial summary of the pipeline in a markdown file. (25 points)


Summary of AI Fairness 360 


As machine learning models become more widely used in domains such as mortgage lending, hiring and prison sentencing, fairness and bias are becoming central foci of the algorithms used in these models. To address issues related to fairness, AI Fairness 360 (AIF360) was developed to help better understand fairness metrics and mitigation techniques.

favorable label: value corresponds to an advantageous outcome
protected attribute: partitions a population into groups with different amounts of benefits, ie) race, gender, caste, religion
privileged: group that has historically been at a systematic advantage
group fairness: groups defined by protected attributes receive similar treatments or outcomes
individual fairness: similar individuals receiving similar treatments or outcomes
bias: systematic error that gives privileged groups advantages and vice versa for unprivileged groups
fairness metric: quantification of unwanted bias in training data or models
bias mitigation algorithm: used to reduce unwanted bias in training data or models

Many other toolkits exist, although they are not as comprehensive as AIF360 in that AIF 360 provides bisa metrics, mitigation algorithms, bias metric explanations, and industrial usability. 

AIF 360 is an end-to-end workflow that allows users to transform raw data to a model, adding in functionality throughout. The general pipeline for bias mitigation works as follows: outputs are new datasets sharing the same protected attributes as other datasets, transitions are transformations that modify features and/or labels, and trapezoids are learned models. One can reach a fair prediction through one of three bias mitigation algorithms: fair pre-processing, fair in-processing, fair post-processing. The three methods each have their nuances, so one must decide which they would like to use based on its respective features.

The Dataset class handles all forms of data (like training data and testing data), and organizes the data by associating each instance of data with protected attributes. It also contains subclasses that are more specific and further organize the data. These classes serve to provide a common structure for the entire pipeline to use and also allow different utility functions and capabilities (from Python) to be applied to the data. 

The Metrics class and its subclasses calculate fairness matrics and check for biases in datasets and models. Because fairness can be defined in many ways and is dependent on the context, many different fairness metrics are included in the toolkit, which gives the user the option to select the best metrics given the data. 

The Explainer class is intended to be associated with the Metrics class.  It is used to provide further insights on computed fairness metrics.  The Explainer class stresses the need for explanations.  Future methodologies may include actionable recourse analysis, counterfactual fairness, and fine-grained localization

# Step 2

Execute the credit decision pipeline that is detecting age bias and removing using the reweighting algorithm and explain the findings. (25 points) -- Executed Below

In [0]:

#!pip install aif360
#!pip install IPython

# Load all necessary packages
import sys
sys.path.insert(1, "../")  

import numpy as np
np.random.seed(0)

from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

from IPython.display import Markdown, display


dataset_orig = GermanDataset(
    protected_attribute_names=['age'],           # this dataset also contains protected
                                                 # attribute for "sex" which we do not
                                                 # consider in this evaluation
    privileged_classes=[lambda x: x >= 25],      # age >=25 is considered privileged
    features_to_drop=['personal_status', 'sex'] # ignore sex-related attributes
)

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#USING REWEIGHTING ALGORITHM

RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Original training dataset

#### Transformed training dataset

[OUTPUT]:
Original training dataset:
Difference in mean outcomes between unprivileged and privileged groups = -0.169905
Transformed training dataset:
Difference in mean outcomes between unprivileged and privileged groups = 0.000000

Findings:
Protected Attribute: Age
Attributes not accounted for: Sex & Personal Status


# Step 3

Apply a different method for detecting and removing bias. (25 points)

In [0]:
#Need to apply a different method than the Reweighting Function which is built-in 

RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

Reweighing.py Source Code Below:

In [0]:
import numpy as np

from aif360.algorithms import Transformer
from aif360.metrics import utils


class Reweighing(Transformer):
    """Reweighing is a preprocessing technique that Weights the examples in each
    (group, label) combination differently to ensure fairness before
    classification [4]_.
    References:
        .. [4] F. Kamiran and T. Calders,  "Data Preprocessing Techniques for
           Classification without Discrimination," Knowledge and Information
           Systems, 2012.
    """

    def __init__(self, unprivileged_groups, privileged_groups):
        """
        Args:
            unprivileged_groups (list(dict)): Representation for unprivileged
                group.
            privileged_groups (list(dict)): Representation for privileged group.
        """
        super(Reweighing, self).__init__(
            unprivileged_groups=unprivileged_groups,
            privileged_groups=privileged_groups)

        self.unprivileged_groups = unprivileged_groups
        self.privileged_groups = privileged_groups

        self.w_p_fav = 1.
        self.w_p_unfav = 1.
        self.w_up_fav = 1.
        self.w_up_unfav = 1.

    def fit(self, dataset):
        """Compute the weights for reweighing the dataset.
        Args:
            dataset (BinaryLabelDataset): Dataset containing true labels.
        Returns:
            Reweighing: Returns self.
        """

        (priv_cond, unpriv_cond, fav_cond, unfav_cond,
        cond_p_fav, cond_p_unfav, cond_up_fav, cond_up_unfav) =\
                self._obtain_conditionings(dataset)

        n = np.sum(dataset.instance_weights, dtype=np.float64)
        n_p = np.sum(dataset.instance_weights[priv_cond], dtype=np.float64)
        n_up = np.sum(dataset.instance_weights[unpriv_cond], dtype=np.float64)
        n_fav = np.sum(dataset.instance_weights[fav_cond], dtype=np.float64)
        n_unfav = np.sum(dataset.instance_weights[unfav_cond], dtype=np.float64)

        n_p_fav = np.sum(dataset.instance_weights[cond_p_fav], dtype=np.float64)
        n_p_unfav = np.sum(dataset.instance_weights[cond_p_unfav],
                           dtype=np.float64)
        n_up_fav = np.sum(dataset.instance_weights[cond_up_fav],
                          dtype=np.float64)
        n_up_unfav = np.sum(dataset.instance_weights[cond_up_unfav],
                            dtype=np.float64)

        # reweighing weights
        self.w_p_fav = n_fav*n_p / (n*n_p_fav)
        self.w_p_unfav = n_unfav*n_p / (n*n_p_unfav)
        self.w_up_fav = n_fav*n_up / (n*n_up_fav)
        self.w_up_unfav = n_unfav*n_up / (n*n_up_unfav)

        return self

    def transform(self, dataset):
        """Transform the dataset to a new dataset based on the estimated
        transformation.
        Args:
            dataset (BinaryLabelDataset): Dataset that needs to be transformed.
        Returns:
            dataset (BinaryLabelDataset): Dataset with transformed
                instance_weights attribute.
        """

        dataset_transformed = dataset.copy(deepcopy=True)

        (_, _, _, _, cond_p_fav, cond_p_unfav, cond_up_fav, cond_up_unfav) =\
                            self._obtain_conditionings(dataset)

        # apply reweighing
        dataset_transformed.instance_weights[cond_p_fav] *= self.w_p_fav
        dataset_transformed.instance_weights[cond_p_unfav] *= self.w_p_unfav
        dataset_transformed.instance_weights[cond_up_fav] *= self.w_up_fav
        dataset_transformed.instance_weights[cond_up_unfav] *= self.w_up_unfav

        return dataset_transformed

##############################
#### Supporting functions ####
##############################
    def _obtain_conditionings(self, dataset):
        """Obtain the necessary conditioning boolean vectors to compute
        instance level weights.
        """
        # conditioning
        priv_cond = utils.compute_boolean_conditioning_vector(
                            dataset.protected_attributes,
                            dataset.protected_attribute_names,
                            condition=self.privileged_groups)
        unpriv_cond = utils.compute_boolean_conditioning_vector(
                            dataset.protected_attributes,
                            dataset.protected_attribute_names,
                            condition=self.unprivileged_groups)
        fav_cond = dataset.labels.ravel() == dataset.favorable_label
        unfav_cond = dataset.labels.ravel() == dataset.unfavorable_label

        # combination of label and privileged/unpriv. groups
        cond_p_fav = np.logical_and(fav_cond, priv_cond)
        cond_p_unfav = np.logical_and(unfav_cond, priv_cond)
        cond_up_fav = np.logical_and(fav_cond, unpriv_cond)
        cond_up_unfav = np.logical_and(unfav_cond, unpriv_cond)

        return (priv_cond, unpriv_cond, fav_cond, unfav_cond,
            cond_p_fav, cond_p_unfav, cond_up_fav, cond_up_unfav)

In [0]:

#!pip install aif360
#!pip install IPython
#!pip install BlackBoxAuditing

# Load all necessary packages
import sys
sys.path.insert(1, "../")  

import numpy as np
np.random.seed(0)

from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.preprocessing import LFR
from aif360.algorithms.preprocessing import DisparateImpactRemover


from IPython.display import Markdown, display


dataset_orig = GermanDataset(
    protected_attribute_names=['age'],           # this dataset also contains protected
                                                 # attribute for "sex" which we do not
                                                 # consider in this evaluation
    privileged_classes=[lambda x: x >= 25],      # age >=25 is considered privileged
    features_to_drop=['personal_status', 'sex'] # ignore sex-related attributes
)

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#USING REWEIGHTING ALGORITHMS
#
#RW = Reweighing(unprivileged_groups=unprivileged_groups,
#                privileged_groups=privileged_groups)
#dataset_transf_train = RW.fit_transform(dataset_orig_train)
#
#metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
#                                               unprivileged_groups=unprivileged_groups,
#                                               privileged_groups=privileged_groups)
#display(Markdown("#### Transformed training dataset"))
#print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#-------------------------------------------------------------------------------
#Disparate Impact Remover PreProcessor Method  
DIR = DisparateImpactRemover(repair_level = 1.0)

dataset_transf_DIR = DIR.fit_transform(dataset_orig_train)
dataset_transf_DIR_train, dataset_transf_DR_test, = dataset_transf_DIR.split([0.7], shuffle=True)

metric_DIR_train = BinaryLabelDatasetMetric(dataset_transf_train, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

display(Markdown("#### Transformed training dataset using Disparate Impact Remover PreProcessor Method"))

print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_DIR_train.mean_difference())


#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.169905


#### Transformed training dataset using Disparate Impact Remover PreProcessor Method

Difference in mean outcomes between unprivileged and privileged groups = 0.000000


# Step 4

Write a thorough explanation comparing the reweighting and the method you selected. (25 points)